# the introduction of  my final capstone project 

## My idea of the creative project(the major analyzing is preparing for week 5)

### In the third week of the connection, I learned how to cluster the city's neighborhoods. Below I will perform a creative analysis. My analysis will focus on Queens in New York, focusing on the distribution of coffee shops here. The relationship between the income of the coffee shop here and the factors such as foot traffic and traffic conditions.

#### First extract the neighborhoods of Queens from New York City's overall data

#### Using FourSquare API to extract coffee shops in Queens

#### Analyze the flow of people and traffic near coffee shops

#### Use clustering methods to classify them according to observations

In [130]:
import time as t
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy import interpolate
from scipy.stats import norm

In [129]:
import time as t
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy import interpolate
from scipy.stats import norm

In [2]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [53]:
pd.set_option('display.max_columns', 50)
#显示所有行
pd.set_option('display.max_rows', 50)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [10]:
neighborhoods_data = newyork_data['features']

In [12]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [13]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [14]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [15]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [24]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631


In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


In [30]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[40.714270, -74.005970], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [39]:
neighborhoods.Borough.value_counts()

Queens           81
Brooklyn         70
Staten Island    63
Bronx            52
Manhattan        40
Name: Borough, dtype: int64

In [63]:
Location_of_Boroughs = neighborhoods.groupby(['Borough']).mean()

In [78]:
Location_of_Boroughs

,Latitude,Longitude
Borough,,
Bronx,40.850482,-73.873442
Brooklyn,40.651089,-73.952999
Manhattan,40.762438,-73.975329
Queens,40.706424,-73.824131
Staten Island,40.588851,-74.137414


In [74]:
Queens_Neighborhoods_Latitude = Location_of_Boroughs.loc['Queens']['Latitude']

In [75]:
Queens_Neighborhoods_Longitude = Location_of_Boroughs.loc['Queens']['Longitude']

In [55]:
Queens_Neighborhoods = neighborhoods[neighborhoods['Borough']=='Queens'].reset_index(drop=True)

In [56]:
Queens_Neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138
...,...,...,...,...
76,Queens,Middle Village,40.716415,-73.881143
77,Queens,Malba,40.790602,-73.826678
78,Queens,Hammels,40.587338,-73.805530
79,Queens,Bayswater,40.611322,-73.765968


In [60]:
print('Queen Borough has {} neighborhoods.'.format(
        Queens_Neighborhoods.shape[0]))

Queen Borough has 81 neighborhoods.


In [81]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[Queens_Neighborhoods_Latitude, Queens_Neighborhoods_Longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Queens_Neighborhoods['Latitude'], Queens_Neighborhoods['Longitude'], Queens_Neighborhoods['Borough'], Queens_Neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffb6c1',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [99]:
CLIENT_ID = 'UFJT32X1CYJBO0AYJIC4353FMP3HMFE5OY10CW4R12C1SHC1'
CLIENT_SECRET = 'AXBMNJOLIG25L2UESSXBGXB3G1H1M5FQLGRNEKVFVAIMWVQK'
VERSION = '20200101' # Foursquare API version
radius = 500
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFJT32X1CYJBO0AYJIC4353FMP3HMFE5OY10CW4R12C1SHC1
CLIENT_SECRET:AXBMNJOLIG25L2UESSXBGXB3G1H1M5FQLGRNEKVFVAIMWVQK


In [88]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='UFJT32X1CYJBO0AYJIC4353FMP3HMFE5OY10CW4R12C1SHC1',
client_secret='AXBMNJOLIG25L2UESSXBGXB3G1H1M5FQLGRNEKVFVAIMWVQK',
v='20200101',
ll='40.7243,-74.0018',
query='coffee',
limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [102]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=restaurant'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Queens_Neighborhoods_Latitude, 
    Queens_Neighborhoods_Longitude, 
    3000, 
    30)

In [104]:
restaurant_results = requests.get(url).json()
restaurant_results

{'meta': {'code': 200, 'requestId': '5e7f2dd4be61c9001cf67a1e'},
 'response': {'venues': [{'id': '514cb2f3e4b07ece6ded65b5',
    'name': "D'Angelo's Italian Pizzeria & Restaurant",
    'location': {'address': '118-07 Queens Blvd',
     'crossStreet': '78th Avenue',
     'lat': 40.71507665112627,
     'lng': -73.83195743247396,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71507665112627,
       'lng': -73.83195743247396}],
     'distance': 1167,
     'postalCode': '11375',
     'cc': 'US',
     'city': 'Forest Hills',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['118-07 Queens Blvd (78th Avenue)',
      'Forest Hills, NY 11375',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delive

In [121]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [124]:
restaurant_results['response']

{'venues': [{'id': '514cb2f3e4b07ece6ded65b5',
   'name': "D'Angelo's Italian Pizzeria & Restaurant",
   'location': {'address': '118-07 Queens Blvd',
    'crossStreet': '78th Avenue',
    'lat': 40.71507665112627,
    'lng': -73.83195743247396,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.71507665112627,
      'lng': -73.83195743247396}],
    'distance': 1167,
    'postalCode': '11375',
    'cc': 'US',
    'city': 'Forest Hills',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['118-07 Queens Blvd (78th Avenue)',
     'Forest Hills, NY 11375',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'shortName': 'Pizza',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
      'suffix': '.png'},
     'primary': True}],
   'delivery': {'id': '278499',
    'url': 'https://www.seamless.com/menu/dangelos-118-07-queens-blvd-forest-hill

In [126]:
restaurants = restaurant_results['response']['venues']

In [128]:
restaurants[0]

{'id': '514cb2f3e4b07ece6ded65b5',
 'name': "D'Angelo's Italian Pizzeria & Restaurant",
 'location': {'address': '118-07 Queens Blvd',
  'crossStreet': '78th Avenue',
  'lat': 40.71507665112627,
  'lng': -73.83195743247396,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71507665112627,
    'lng': -73.83195743247396}],
  'distance': 1167,
  'postalCode': '11375',
  'cc': 'US',
  'city': 'Forest Hills',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['118-07 Queens Blvd (78th Avenue)',
   'Forest Hills, NY 11375',
   'United States']},
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True}],
 'delivery': {'id': '278499',
  'url': 'https://www.seamless.com/menu/dangelos-118-07-queens-blvd-forest-hills/278499?affiliate=1131&utm_source=foursquare-affiliate-network&

In [119]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Queens_Neighborhoods_Latitude, 
    Queens_Neighborhoods_Longitude,
    2000)

In [120]:
trending_results = requests.get(url).json()
trending_results

{'meta': {'code': 200, 'requestId': '5e7f2ea140a7ea001bbf1287'},
 'response': {'venues': []}}

In [113]:
Queens_Neighborhoods_Latitude

40.70642417479845

In [114]:
Queens_Neighborhoods_Longitude

-73.82413081200184